In [17]:
from time import time
from model import S3VR
import numpy as np
from utils import *
import itertools
from sklearn.svm import SVR
from tqdm import tqdm
from sklearn.model_selection import KFold

In [3]:
labeled_X = np.load('labeled_data.npy')
unlabeled_X = np.load('unlabeled_data.npy')
y = np.load('labels.npy')

labeled_X = labeled_X.astype(np.float64)
unlabeled_X = unlabeled_X.astype(np.float64)
y = y.astype(np.float64)

In [20]:
p = [[0.1, 1, 10, 100],[1e-3, 1e-2, 0.1, 1]]
ps = list(itertools.product(*p))

kf = KFold(n_splits=5)

res = []
for c, e in tqdm(ps):
    temp = 0
    for train, test in kf.split(labeled_X):
        preds = np.zeros_like(y[test, :])
        for i in range(y.shape[1]):
            X_train, X_test, y_train, _ = labeled_X[train], labeled_X[test], y[train, i], y[test, i]
            svr = SVR(C=c, epsilon=e)
            svr.fit(X_train, y_train)
            preds[:, i] = svr.predict(X_test)
        temp += MCRMSE(y[test, :], preds.T)
    res.append(temp / 5)

100%|██████████| 16/16 [08:24<00:00, 31.55s/it]


In [21]:
ps[np.argsort(np.array(res))[0]]

(0.1, 0.1)

In [4]:
s3vr = S3VR(5, 10, 0.5, 10)
s3vr.fit(labeled_X, y[:, 0], unlabeled_X)
preds = s3vr.predict(labeled_X)

# info:

C: 0.1, 1, 10, 100
epsilon: 1e-3, 1e-2, 0.1, 1

k-local:
k-global:
r:
beta: